Install Required Libraries

In [1]:
# Install necessary libraries
!pip install torch torchvision matplotlib Pillow

# Import required libraries
import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os


Prepare Images

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Path to the folder containing the images
image_folder = '/content/drive/MyDrive/fcn_images'

# List the image files
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

# Define transformations for resizing, normalizing, and converting to tensor
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])


Mounted at /content/drive


Load Pre-trained FCN Network

In [4]:
# Load pretrained FCN model
model = models.segmentation.fcn_resnet50(pretrained=True)
model.eval()  # Set model to evaluation mode

# Move model to GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fcn_resnet50_coco-1167a1af.pth" to /root/.cache/torch/hub/checkpoints/fcn_resnet50_coco-1167a1af.pth
100%|██████████| 135M/135M [00:00<00:00, 183MB/s]


Define Helper Functions

In [7]:
def save_feature_maps(output, save_folder, image_name):
    """
    Save the 21 feature maps as tiles in one image, in the same folder as the original images.
    """
    feature_maps = output['out'].squeeze(0).detach().cpu().numpy()  # Extract feature maps
    num_maps = feature_maps.shape[0]
    grid_size = int(np.ceil(np.sqrt(num_maps)))  # Determine grid size for tiles
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))

    for i, ax in enumerate(axes.flat):
        if i < num_maps:
            ax.imshow(feature_maps[i], cmap='viridis')
        ax.axis('off')
    plt.tight_layout()
    save_path = os.path.join(save_folder, f"{image_name}_feature_maps.png")
    plt.savefig(save_path)
    plt.close()

def create_segmentation(output, save_folder, image_name):
    """
    Create a segmentation image where each color represents one class,
    and save it in the same folder as the original images.
    """
    seg_map = torch.argmax(output['out'], dim=1).squeeze(0).cpu().numpy()  # Get class indices
    # Use a random color map for visualization
    colormap = np.random.randint(0, 255, (21, 3), dtype=np.uint8)  # 21 classes
    segmentation = colormap[seg_map]  # Map class indices to colors
    plt.imshow(segmentation)
    plt.axis('off')
    save_path = os.path.join(save_folder, f"{image_name}_segmentation.png")
    plt.savefig(save_path)
    plt.close()


Process each image

In [8]:
for image_file in image_files:
    # Load and preprocess the image
    img = Image.open(image_file).convert('RGB')
    input_tensor = data_transforms(img).unsqueeze(0).to(device)

    # Get the network output
    with torch.no_grad():
        output = model(input_tensor)

    # Save feature maps and segmentation in the same folder as the original images
    image_name = os.path.splitext(os.path.basename(image_file))[0]
    save_folder = os.path.dirname(image_file)  # Get the folder path of the original image
    save_feature_maps(output, save_folder, image_name)
    create_segmentation(output, save_folder, image_name)

    print(f"Processed and saved results for {image_file}")


Processed and saved results for /content/drive/MyDrive/fcn_images/farm_animals.jpg
Processed and saved results for /content/drive/MyDrive/fcn_images/farm_animals2.jpg
Processed and saved results for /content/drive/MyDrive/fcn_images/living_room.jpg
Processed and saved results for /content/drive/MyDrive/fcn_images/living_room2.jpg
Processed and saved results for /content/drive/MyDrive/fcn_images/city_street.jpg
